In [2]:
import time
from sklearn import svm
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import confusion_matrix
import numpy as np
import pickle
import data_manipulation as dm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, normalize
from tsfresh.utilities.dataframe_functions import impute
import ADP

In [4]:
### Loading data analysis result 

with open('data_analysis.pkl', 'rb') as fp:
    data_analysis = pickle.load(fp)

data_clouds_dic = data_analysis['data_clouds_dic']

In [8]:
data_clouds_dic['iteration_0']['granularity_1']['data_cloud_0'].keys()

dict_keys(['data', 'target'])

In [10]:
### Make detections with the scikit methods

# Example settings
signal_fraction = 0.01

names = ["One-Class SVM", "Isolation Forest", "Local Outlier Factor"]
anomaly_algorithms = [svm.OneClassSVM(nu=signal_fraction, kernel="rbf", gamma=0.1),
                      IsolationForest(contamination=signal_fraction, random_state=42),
                      LocalOutlierFactor(n_neighbors=3, contamination=signal_fraction)]
accuracy_dict = {}
for it in data_clouds_dic:
    aux = {}
    for gra in data_clouds_dic[it]:
        aux2 = {}
        for dc in data_clouds_dic[it][gra]:
            X = data_clouds_dic[it][gra][dc]['data']
            Y = data_clouds_dic[it][gra][dc]['target']
            models_dict = {}
            for name, algorithm in zip(names,anomaly_algorithms):
                aux3 = {}
                t0 = time.time()

                if np.shape(X)[0] != 1:
                    y_pred = algorithm.fit_predict(X)
                    tn, fp, fn, tp = confusion_matrix(Y, y_pred, labels=[1,-1]).ravel()
                    t1 = time.time()    

                    aux3["True_Positive"] = tp
                    aux3["True_Negative"] = tn
                    aux3["False_Positive"] = fp
                    aux3["False_Negative"] = fn
                    aux3["Time"] = t1-t0
                    models_dict[name] = aux3

                else:
                    models_dict['mono_event'] = 1
            aux2[dc] = models_dict
        aux[gra] = aux2
    accuracy_dict[it] = aux

/home/thiago/anaconda3/lib/python3.7/site-packages/sklearn/neighbors/_lof.py:277: UserWarning: n_neighbors (3) is greater than the total number of samples (2). n_neighbors will be set to (n_samples - 1) for estimation.
  % (self.n_neighbors, n_samples))
/home/thiago/anaconda3/lib/python3.7/site-packages/sklearn/neighbors/_lof.py:277: UserWarning: n_neighbors (3) is greater than the total number of samples (2). n_neighbors will be set to (n_samples - 1) for estimation.
  % (self.n_neighbors, n_samples))
/home/thiago/anaconda3/lib/python3.7/site-packages/sklearn/neighbors/_lof.py:277: UserWarning: n_neighbors (3) is greater than the total number of samples (2). n_neighbors will be set to (n_samples - 1) for estimation.
  % (self.n_neighbors, n_samples))
/home/thiago/anaconda3/lib/python3.7/site-packages/sklearn/neighbors/_lof.py:277: UserWarning: n_neighbors (3) is greater than the total number of samples (2). n_neighbors will be set to (n_samples - 1) for estimation.
  % (self.n_neighbo

/home/thiago/anaconda3/lib/python3.7/site-packages/sklearn/neighbors/_lof.py:277: UserWarning: n_neighbors (3) is greater than the total number of samples (2). n_neighbors will be set to (n_samples - 1) for estimation.
  % (self.n_neighbors, n_samples))
/home/thiago/anaconda3/lib/python3.7/site-packages/sklearn/neighbors/_lof.py:277: UserWarning: n_neighbors (3) is greater than the total number of samples (2). n_neighbors will be set to (n_samples - 1) for estimation.
  % (self.n_neighbors, n_samples))
/home/thiago/anaconda3/lib/python3.7/site-packages/sklearn/neighbors/_lof.py:277: UserWarning: n_neighbors (3) is greater than the total number of samples (2). n_neighbors will be set to (n_samples - 1) for estimation.
  % (self.n_neighbors, n_samples))
/home/thiago/anaconda3/lib/python3.7/site-packages/sklearn/neighbors/_lof.py:277: UserWarning: n_neighbors (3) is greater than the total number of samples (2). n_neighbors will be set to (n_samples - 1) for estimation.
  % (self.n_neighbo

/home/thiago/anaconda3/lib/python3.7/site-packages/sklearn/neighbors/_lof.py:277: UserWarning: n_neighbors (3) is greater than the total number of samples (2). n_neighbors will be set to (n_samples - 1) for estimation.
  % (self.n_neighbors, n_samples))
/home/thiago/anaconda3/lib/python3.7/site-packages/sklearn/neighbors/_lof.py:277: UserWarning: n_neighbors (3) is greater than the total number of samples (2). n_neighbors will be set to (n_samples - 1) for estimation.
  % (self.n_neighbors, n_samples))
/home/thiago/anaconda3/lib/python3.7/site-packages/sklearn/neighbors/_lof.py:277: UserWarning: n_neighbors (3) is greater than the total number of samples (2). n_neighbors will be set to (n_samples - 1) for estimation.
  % (self.n_neighbors, n_samples))
/home/thiago/anaconda3/lib/python3.7/site-packages/sklearn/neighbors/_lof.py:277: UserWarning: n_neighbors (3) is greater than the total number of samples (2). n_neighbors will be set to (n_samples - 1) for estimation.
  % (self.n_neighbo

/home/thiago/anaconda3/lib/python3.7/site-packages/sklearn/neighbors/_lof.py:277: UserWarning: n_neighbors (3) is greater than the total number of samples (2). n_neighbors will be set to (n_samples - 1) for estimation.
  % (self.n_neighbors, n_samples))
/home/thiago/anaconda3/lib/python3.7/site-packages/sklearn/neighbors/_lof.py:277: UserWarning: n_neighbors (3) is greater than the total number of samples (2). n_neighbors will be set to (n_samples - 1) for estimation.
  % (self.n_neighbors, n_samples))
/home/thiago/anaconda3/lib/python3.7/site-packages/sklearn/neighbors/_lof.py:277: UserWarning: n_neighbors (3) is greater than the total number of samples (2). n_neighbors will be set to (n_samples - 1) for estimation.
  % (self.n_neighbors, n_samples))
/home/thiago/anaconda3/lib/python3.7/site-packages/sklearn/neighbors/_lof.py:277: UserWarning: n_neighbors (3) is greater than the total number of samples (2). n_neighbors will be set to (n_samples - 1) for estimation.
  % (self.n_neighbo

/home/thiago/anaconda3/lib/python3.7/site-packages/sklearn/neighbors/_lof.py:277: UserWarning: n_neighbors (3) is greater than the total number of samples (2). n_neighbors will be set to (n_samples - 1) for estimation.
  % (self.n_neighbors, n_samples))
/home/thiago/anaconda3/lib/python3.7/site-packages/sklearn/neighbors/_lof.py:277: UserWarning: n_neighbors (3) is greater than the total number of samples (2). n_neighbors will be set to (n_samples - 1) for estimation.
  % (self.n_neighbors, n_samples))
/home/thiago/anaconda3/lib/python3.7/site-packages/sklearn/neighbors/_lof.py:277: UserWarning: n_neighbors (3) is greater than the total number of samples (2). n_neighbors will be set to (n_samples - 1) for estimation.
  % (self.n_neighbors, n_samples))
/home/thiago/anaconda3/lib/python3.7/site-packages/sklearn/neighbors/_lof.py:277: UserWarning: n_neighbors (3) is greater than the total number of samples (2). n_neighbors will be set to (n_samples - 1) for estimation.
  % (self.n_neighbo

/home/thiago/anaconda3/lib/python3.7/site-packages/sklearn/neighbors/_lof.py:277: UserWarning: n_neighbors (3) is greater than the total number of samples (2). n_neighbors will be set to (n_samples - 1) for estimation.
  % (self.n_neighbors, n_samples))
/home/thiago/anaconda3/lib/python3.7/site-packages/sklearn/neighbors/_lof.py:277: UserWarning: n_neighbors (3) is greater than the total number of samples (2). n_neighbors will be set to (n_samples - 1) for estimation.
  % (self.n_neighbors, n_samples))
/home/thiago/anaconda3/lib/python3.7/site-packages/sklearn/neighbors/_lof.py:277: UserWarning: n_neighbors (3) is greater than the total number of samples (2). n_neighbors will be set to (n_samples - 1) for estimation.
  % (self.n_neighbors, n_samples))
/home/thiago/anaconda3/lib/python3.7/site-packages/sklearn/neighbors/_lof.py:277: UserWarning: n_neighbors (3) is greater than the total number of samples (2). n_neighbors will be set to (n_samples - 1) for estimation.
  % (self.n_neighbo

/home/thiago/anaconda3/lib/python3.7/site-packages/sklearn/neighbors/_lof.py:277: UserWarning: n_neighbors (3) is greater than the total number of samples (2). n_neighbors will be set to (n_samples - 1) for estimation.
  % (self.n_neighbors, n_samples))
/home/thiago/anaconda3/lib/python3.7/site-packages/sklearn/neighbors/_lof.py:277: UserWarning: n_neighbors (3) is greater than the total number of samples (2). n_neighbors will be set to (n_samples - 1) for estimation.
  % (self.n_neighbors, n_samples))
/home/thiago/anaconda3/lib/python3.7/site-packages/sklearn/neighbors/_lof.py:277: UserWarning: n_neighbors (3) is greater than the total number of samples (2). n_neighbors will be set to (n_samples - 1) for estimation.
  % (self.n_neighbors, n_samples))
/home/thiago/anaconda3/lib/python3.7/site-packages/sklearn/neighbors/_lof.py:277: UserWarning: n_neighbors (3) is greater than the total number of samples (2). n_neighbors will be set to (n_samples - 1) for estimation.
  % (self.n_neighbo

/home/thiago/anaconda3/lib/python3.7/site-packages/sklearn/neighbors/_lof.py:277: UserWarning: n_neighbors (3) is greater than the total number of samples (2). n_neighbors will be set to (n_samples - 1) for estimation.
  % (self.n_neighbors, n_samples))
/home/thiago/anaconda3/lib/python3.7/site-packages/sklearn/neighbors/_lof.py:277: UserWarning: n_neighbors (3) is greater than the total number of samples (2). n_neighbors will be set to (n_samples - 1) for estimation.
  % (self.n_neighbors, n_samples))
/home/thiago/anaconda3/lib/python3.7/site-packages/sklearn/neighbors/_lof.py:277: UserWarning: n_neighbors (3) is greater than the total number of samples (2). n_neighbors will be set to (n_samples - 1) for estimation.
  % (self.n_neighbors, n_samples))
/home/thiago/anaconda3/lib/python3.7/site-packages/sklearn/neighbors/_lof.py:277: UserWarning: n_neighbors (3) is greater than the total number of samples (2). n_neighbors will be set to (n_samples - 1) for estimation.
  % (self.n_neighbo

/home/thiago/anaconda3/lib/python3.7/site-packages/sklearn/neighbors/_lof.py:277: UserWarning: n_neighbors (3) is greater than the total number of samples (2). n_neighbors will be set to (n_samples - 1) for estimation.
  % (self.n_neighbors, n_samples))
/home/thiago/anaconda3/lib/python3.7/site-packages/sklearn/neighbors/_lof.py:277: UserWarning: n_neighbors (3) is greater than the total number of samples (2). n_neighbors will be set to (n_samples - 1) for estimation.
  % (self.n_neighbors, n_samples))
/home/thiago/anaconda3/lib/python3.7/site-packages/sklearn/neighbors/_lof.py:277: UserWarning: n_neighbors (3) is greater than the total number of samples (2). n_neighbors will be set to (n_samples - 1) for estimation.
  % (self.n_neighbors, n_samples))
/home/thiago/anaconda3/lib/python3.7/site-packages/sklearn/neighbors/_lof.py:277: UserWarning: n_neighbors (3) is greater than the total number of samples (2). n_neighbors will be set to (n_samples - 1) for estimation.
  % (self.n_neighbo

/home/thiago/anaconda3/lib/python3.7/site-packages/sklearn/neighbors/_lof.py:277: UserWarning: n_neighbors (3) is greater than the total number of samples (2). n_neighbors will be set to (n_samples - 1) for estimation.
  % (self.n_neighbors, n_samples))
/home/thiago/anaconda3/lib/python3.7/site-packages/sklearn/neighbors/_lof.py:277: UserWarning: n_neighbors (3) is greater than the total number of samples (2). n_neighbors will be set to (n_samples - 1) for estimation.
  % (self.n_neighbors, n_samples))
/home/thiago/anaconda3/lib/python3.7/site-packages/sklearn/neighbors/_lof.py:277: UserWarning: n_neighbors (3) is greater than the total number of samples (2). n_neighbors will be set to (n_samples - 1) for estimation.
  % (self.n_neighbors, n_samples))
/home/thiago/anaconda3/lib/python3.7/site-packages/sklearn/neighbors/_lof.py:277: UserWarning: n_neighbors (3) is greater than the total number of samples (2). n_neighbors will be set to (n_samples - 1) for estimation.
  % (self.n_neighbo

In [16]:
accuracy_dict['iteration_0']['granularity_1']['data_cloud_0'].keys()

dict_keys(['One-Class SVM', 'Isolation Forest', 'Local Outlier Factor'])